In [ ]:
import time
import glob
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from matplotlib import pyplot as plt
from matplotlib import image as mpimg

from tqdm.notebook import tqdm
import seaborn as sns
import koreanize_matplotlib


In [ ]:
options = webdriver.ChromeOptions()

prefs = {'download.default_directory':'/home/han/Han_ws/eda/data/gas_station/',
         'download.prompt_for_download':False}
options.add_experimental_option('prefs', prefs)


In [ ]:

url = "https://www.opinet.co.kr/searRgSelect.do"
driver = webdriver.Chrome(service=Service("../driver/chromedriver"), options=options)
driver.get(url)

In [ ]:
driver.quit()

In [ ]:
sido = driver.find_element(By.ID, "SIDO_NM0")
sido_list = sido.find_elements(By.TAG_NAME, "option")
sido_names = [sido_name.get_attribute("value") for sido_name in sido_list]
sido_names = sido_names[1:]
sido_names

In [ ]:
gu = driver.find_element(By.ID, "SIGUNGU_NM0")
gu_list = gu.find_elements(By.TAG_NAME,"option")
gu_names = [gu_name.get_attribute("value") for gu_name in gu_list]
gu_names = gu_names[1:]
gu.send_keys(gu_names[5])

In [ ]:
gu_names

In [ ]:
for gu_name in tqdm(gu_names):
    gu = driver.find_element(By.ID, "SIGUNGU_NM0")
    gu.send_keys(gu_name)

    save = WebDriverWait(driver, timeout=30).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_type6_ex_save")))
    ActionChains(driver).move_to_element(save).perform()
    save.send_keys(Keys.ENTER)

    driver.implicitly_wait(3)


In [ ]:
len(glob.glob("../data/gas_station/지역_*"))

In [ ]:
driver.quit()

In [ ]:
import pandas as pd

file_list=glob.glob("../data/gas_station/지역_*")
data = pd.read_excel(file_list[0], header=2)

data_list = []
for file in file_list:
    data = pd.read_excel(file, header=2)
    data_list.append(data)

oil_data = pd.concat(data_list)
oil_data = pd.DataFrame({"상호": oil_data["상호"],
                         "주소": oil_data["주소"],
                         "가격": oil_data["휘발유"],
                         "셀프": oil_data["셀프여부"],
                         "상표": oil_data["상표"]
                         })
oil_data.reset_index(drop=True, inplace=True)
oil_data["구"] = [address.split()[1] for address in oil_data["주소"]]
oil_data = oil_data[oil_data["가격"] != "-"]
oil_data["가격"] = oil_data["가격"].astype("float")
oil_data.reset_index(drop=True, inplace=True)


In [ ]:
oil_data.boxplot(column="가격", by="셀프", figsize=(12,8));

In [ ]:


price_data = oil_data[oil_data["셀프"] == "Y"]
price_data = price_data["가격"]
price_data.median()


oil_data.boxplot.text(["Y"],
              price_data,
              price_data,
              fontsize=15)
        

In [ ]:
oil_data